In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from functools import partial
import geopy
import time
from geopy.extra.rate_limiter import RateLimiter
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [ ]:
URL = "https://www.caproasia.com/2023/06/21/2023-top-20-most-expensive-cities-in-the-world-top-10-cities-are-new-york-hong-kong-geneva-london-singapore-zurich-san-francisco-tel-aviv-seoul-tokyo/"

In [ ]:
# Header for request
HEADER = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en; q=0.5'})
geolocator = Nominatim(user_agent="geocode_locations_from_webpage")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5, max_retries=3)
geo_key = 'e0e73007ebe240c6836e278bbc510da2'
geocoder = OpenCageGeocode(geo_key)

In [ ]:
webpage = requests.get(URL, headers=HEADER)
webpage

In [ ]:
soup = BeautifulSoup(webpage.content, "html.parser")
#soup_4

In [ ]:
places = soup.find_all("td", attrs = {'style': 'height: 23px; width: 28.784648%;'})
places

In [ ]:
location_description = geocoder.geocode(places[2].text)
#location_description

In [ ]:
city_address = []
for i in range(20):
    
    # GETTING RAW DATA
    try:
        location_description = geocoder.geocode(places[i].text)
    except Exception:
        print(str(i) + " " + "No data found for location: " + places[i].text)
        continue
    
    # GETTING ADDRESS VALUES
    latitude = str(location_description[0]['geometry']['lat'])
    longitude = str(location_description[0]['geometry']['lng'])
    city = places[i].text
    country = location_description[0]['components']['country']
    country_code = location_description[0]['components']['country_code']
    continent = location_description[0]['components']['continent']
    
    #GETTING POSTCODE
    lat_long = str(latitude) + ","+ str(longitude) 
    location2 = geolocator.reverse(lat_long, language="en")
    try:
        postcode = location2.raw['address']['postcode'].replace(" ","")
    except:
        postcode = "NULL"
        
        
    # ADDING VARIABLES TO DICTIONARY
    #print(str(i) +"  "+ city +"  " + country + "  " + country_code.upper() + "  " + continent + "  " + latitude + "  "+ longitude+ "  " + postcode)
    address_dict = {'City': city, 'Country': country, 'Country Code':country_code.upper(), 'Continent': continent, 'Latitude': str(round(float(latitude),2)), 'Longitude': str(round(float(longitude),2)), 'Postcode': postcode}
    city_address.append(address_dict)
    
print(city_address)

In [ ]:
ca_most_expensive_df = pd.DataFrame.from_dict(city_address)
ca_most_expensive_df